In [1]:
%%html
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
.widget-label {
    font-weight: bold !important;
    color: #1f77b4 !important;
}
.success { color: green; font-weight: bold; }
.error { color: red; font-weight: bold; }
.warning { color: orange; font-weight: bold; }
</style>

In [2]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from scipy.fftpack import dct, idct
import pywt
from skimage.metrics import structural_similarity as ssim
import warnings
import ipywidgets as widgets
from IPython.display import display, clear_output
import json
import datetime
import hashlib
import os
from io import BytesIO
import base64

warnings.filterwarnings('ignore')

print("🔍 Enhanced DWT+DCT+SVD Watermarking System with Copyright Management")

🔍 Enhanced DWT+DCT+SVD Watermarking System with Copyright Management


In [3]:
class RobustWatermarkingSystem:
    def __init__(self, wavelet='haar', block_size=8, base_alpha=0.2):
        """
        Completely revised robust watermarking system
        """
        self.wavelet = wavelet
        self.block_size = block_size
        self.base_alpha = base_alpha
        self.database = {}
        self.verification_log = []
        
    def generate_copyright_info(self, owner_name, contact_info, rights="All rights reserved"):
        """Generate copyright information"""
        timestamp = datetime.datetime.now().isoformat()
        copyright_id = hashlib.md5(f"{owner_name}{timestamp}".encode()).hexdigest()[:16]
        
        return {
            'copyright_id': copyright_id,
            'owner_name': owner_name,
            'contact_info': contact_info,
            'rights': rights,
            'timestamp': timestamp,
            'registration_date': datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
    
    def create_robust_watermark(self, copyright_id, owner_name, size=(64, 64)):
        """Create a robust, distinctive watermark pattern"""
        watermark = np.zeros(size, dtype=np.float64)
        
        # Create a unique pattern based on copyright info
        pattern_seed = hashlib.md5(f"{copyright_id}{owner_name}".encode()).hexdigest()
        np.random.seed(int(pattern_seed[:8], 16) % 2**32)
        
        # Create a pseudo-random but reproducible pattern
        for i in range(0, size[0], 8):
            for j in range(0, size[1], 8):
                if np.random.random() > 0.5:
                    watermark[i:i+4, j:j+4] = 1.0
                if np.random.random() > 0.7:
                    watermark[i+4:i+8, j+4:j+8] = 0.7
        
        return watermark
    
    def preprocess_image(self, image_path, target_size=(512, 512)):
        """Preprocess image for watermarking"""
        img = np.array(Image.open(image_path).convert('L'))
        img = cv2.resize(img, target_size)
        return img.astype(np.float64) / 255.0
    
    def calculate_complexity(self, image):
        """Calculate image complexity for adaptive watermarking"""
        edges = cv2.Canny((image * 255).astype(np.uint8), 50, 150)
        return np.sum(edges > 0) / edges.size
    
    def adaptive_alpha(self, complexity):
        """Adaptive watermark strength"""
        return min(self.base_alpha * (1 + complexity), 0.3)
    
    def embed_watermark_robust(self, host_image, watermark_image, alpha):
        """
        Robust watermark embedding with proper signal isolation
        """
        print("   🔧 Embedding watermark robustly...")
        
        # Store original host for verification
        original_host = host_image.copy()
        
        # Apply 2-level DWT to host
        coeffs_h2 = pywt.wavedec2(host_image, self.wavelet, level=2)
        LL2, (HL2, LH2, HH2), (HL1, LH1, HH1) = coeffs_h2
        
        # Resize watermark to match LL2 subband size
        wm_resized = cv2.resize(watermark_image, LL2.shape[::-1])
        
        # Apply DCT to host LL2 subband
        LL2_dct = dct(dct(LL2.T, norm='ortho').T, norm='ortho')
        
        # Apply SVD to host DCT coefficients
        U_h, S_h, Vt_h = np.linalg.svd(LL2_dct, full_matrices=False)
        
        # Apply DCT + SVD to watermark
        wm_dct = dct(dct(wm_resized.T, norm='ortho').T, norm='ortho')
        U_w, S_w, Vt_w = np.linalg.svd(wm_dct, full_matrices=False)
        
        # CRITICAL FIX: Store original host SVD for proper extraction
        S_h_original = S_h.copy()
        
        # Embed watermark with proper scaling
        min_size = min(len(S_h), len(S_w))
        scaling_factor = np.mean(S_h[:min_size]) / np.mean(S_w[:min_size]) if np.mean(S_w[:min_size]) > 0 else 1.0
        scaled_S_w = S_w[:min_size] * scaling_factor * 0.1  # Reduced impact
        
        S_embedded = S_h.copy()
        S_embedded[:min_size] = S_h[:min_size] + alpha * scaled_S_w
        
        # Reconstruct watermarked coefficients
        LL2_dct_embedded = U_h @ np.diag(S_embedded) @ Vt_h
        LL2_embedded = idct(idct(LL2_dct_embedded.T, norm='ortho').T, norm='ortho')
        
        # Reconstruct complete image
        coeffs_embedded = [LL2_embedded, (HL2, LH2, HH2), (HL1, LH1, HH1)]
        watermarked = pywt.waverec2(coeffs_embedded, self.wavelet)
        watermarked = np.clip(watermarked, 0, 1)
        
        # Store extraction data with original host information
        extraction_data = {
            'U_h': U_h, 'Vt_h': Vt_h, 'S_h_original': S_h_original,
            'S_w': S_w, 'alpha': alpha, 'scaling_factor': scaling_factor,
            'min_size': min_size, 'wm_size': wm_resized.shape,
            'wavelet': self.wavelet, 'original_LL2': LL2,  # Store original LL2 for verification
            'host_dct_original': LL2_dct  # Store original DCT
        }
        
        return watermarked, extraction_data
    
    def extract_watermark_robust(self, test_image, extraction_data, original_watermark):
        """
        Robust watermark extraction with proper signal isolation
        """
        try:
            # Apply same DWT decomposition to test image
            coeffs_t2 = pywt.wavedec2(test_image, extraction_data['wavelet'], level=2)
            LL2_t, _, _ = coeffs_t2
            
            # Apply DCT to test LL2 subband
            LL2_t_dct = dct(dct(LL2_t.T, norm='ortho').T, norm='ortho')
            
            # Apply SVD to test DCT coefficients
            U_t, S_t, Vt_t = np.linalg.svd(LL2_t_dct, full_matrices=False)
            
            # Get extraction parameters
            U_h = extraction_data['U_h']
            Vt_h = extraction_data['Vt_h'] 
            S_h_original = extraction_data['S_h_original']
            S_w = extraction_data['S_w']
            alpha = extraction_data['alpha']
            scaling_factor = extraction_data['scaling_factor']
            min_size = extraction_data['min_size']
            
            # Calculate the difference in singular values
            delta_S = S_t - S_h_original
            
            # Extract watermark singular values with proper scaling
            S_w_extracted = (delta_S[:min_size] / alpha) / scaling_factor
            
            # Reconstruct watermark DCT coefficients
            # Use the original watermark's U and V matrices for proper reconstruction
            wm_dct_extracted = U_h[:, :min_size] @ np.diag(S_w_extracted) @ Vt_h[:min_size, :]
            
            # Apply inverse DCT
            wm_extracted = idct(idct(wm_dct_extracted.T, norm='ortho').T, norm='ortho')
            
            # Resize to original watermark size
            target_size = extraction_data['wm_size']
            wm_extracted = cv2.resize(wm_extracted, target_size[::-1])
            wm_extracted = np.clip(wm_extracted, 0, 1)
            
            # CRITICAL: Apply thresholding to clean up the extracted watermark
            # This helps distinguish real watermarks from noise
            threshold = 0.3
            wm_clean = np.zeros_like(wm_extracted)
            wm_clean[wm_extracted > threshold] = 1.0
            
            return wm_clean
            
        except Exception as e:
            print(f"   ❌ Extraction error: {str(e)}")
            return None
    
    def calculate_similarity_robust(self, original_wm, extracted_wm):
        """
        Robust similarity calculation with noise rejection
        """
        if extracted_wm is None:
            return 0.0
            
        if original_wm.shape != extracted_wm.shape:
            extracted_wm = cv2.resize(extracted_wm, original_wm.shape[::-1])
        
        # Calculate binary correlation for robustness
        original_binary = (original_wm > 0.5).astype(np.float64)
        extracted_binary = (extracted_wm > 0.5).astype(np.float64)
        
        # Calculate correlation coefficient
        correlation = np.corrcoef(original_binary.flatten(), extracted_binary.flatten())[0, 1]
        
        # Return normalized correlation (0 to 1)
        return max(0, (correlation + 1) / 2) if not np.isnan(correlation) else 0.0
    
    def register_copyright(self, image_path, owner_name, contact_info, rights="All rights reserved"):
        """Register copyright with robust watermarking"""
        print("📝 Starting robust copyright registration...")
        
        copyright_info = self.generate_copyright_info(owner_name, contact_info, rights)
        
        # Create robust watermark
        watermark_image = self.create_robust_watermark(
            copyright_info['copyright_id'], owner_name
        )
        
        # Preprocess host image
        host_image = self.preprocess_image(image_path)
        
        # Calculate adaptive alpha
        complexity = self.calculate_complexity(host_image)
        alpha = self.adaptive_alpha(complexity)
        
        print(f"📊 Image Analysis:")
        print(f"   Complexity: {complexity:.4f}")
        print(f"   Adaptive Alpha: {alpha:.4f}")
        
        # Embed watermark robustly
        watermarked_image, extraction_data = self.embed_watermark_robust(
            host_image, watermark_image, alpha
        )
        
        # Store in database
        self.database[copyright_info['copyright_id']] = {
            'copyright_info': copyright_info,
            'extraction_data': extraction_data,
            'original_watermark': watermark_image,
            'original_host': host_image,  # Store original for comparison
            'watermarked_image': watermarked_image,
            'original_size': host_image.shape
        }
        
        print("✅ Copyright registration completed!")
        print(f"📄 Copyright ID: {copyright_info['copyright_id']}")
        
        return watermarked_image, copyright_info
    
    def verify_authenticity(self, image_path, expected_owner=None):
        """Verify authenticity with robust extraction"""
        print("🔍 Starting robust authenticity verification...")
        
        test_image = self.preprocess_image(image_path)
        
        best_match = None
        best_similarity = 0
        extracted_watermark = None
        
        for copyright_id, data in self.database.items():
            try:
                # Extract watermark using robust method
                extracted_wm = self.extract_watermark_robust(
                    test_image, 
                    data['extraction_data'],
                    data['original_watermark']
                )
                
                if extracted_wm is not None:
                    # Calculate robust similarity
                    similarity = self.calculate_similarity_robust(
                        data['original_watermark'], extracted_wm
                    )
                    
                    print(f"   🔍 {copyright_id}: similarity = {similarity:.4f}")
                    
                    if similarity > best_similarity:
                        best_similarity = similarity
                        best_match = data['copyright_info']
                        extracted_watermark = extracted_wm
                else:
                    print(f"   ❌ Extraction failed for {copyright_id}")
                    
            except Exception as e:
                print(f"   ⚠️ Error for {copyright_id}: {str(e)}")
                continue
        
        # Generate report with adjusted thresholds
        report = self.generate_verification_report(best_match, best_similarity, expected_owner)
        self.log_verification(image_path, report)
        
        return report, best_match, extracted_watermark
    
    def generate_verification_report(self, copyright_info, similarity, expected_owner):
        """Generate verification report with proper thresholds"""
        # Adjusted thresholds for binary watermark matching
        is_authentic = similarity > 0.6  # Higher threshold for binary patterns
        
        report = {
            'timestamp': datetime.datetime.now().isoformat(),
            'similarity_score': similarity,
            'authentication_status': 'AUTHENTIC' if is_authentic else 'SUSPICIOUS',
            'confidence_level': self.get_confidence_level(similarity),
            'expected_owner_match': False
        }
        
        if copyright_info:
            report.update({
                'copyright_id': copyright_info['copyright_id'],
                'registered_owner': copyright_info['owner_name'],
                'registration_date': copyright_info['registration_date']
            })
            
            if expected_owner and expected_owner.lower() in copyright_info['owner_name'].lower():
                report['expected_owner_match'] = True
        
        return report
    
    def get_confidence_level(self, similarity):
        """Get confidence level for binary watermark matching"""
        if similarity >= 0.8: return "VERY HIGH"
        elif similarity >= 0.7: return "HIGH" 
        elif similarity >= 0.6: return "MEDIUM"
        elif similarity >= 0.4: return "LOW"
        else: return "VERY LOW"
    
    def log_verification(self, image_path, report):
        """Log verification attempt"""
        self.verification_log.append({
            'image_path': image_path,
            'timestamp': report['timestamp'],
            'status': report['authentication_status'],
            'similarity': report['similarity_score'],
            'confidence': report['confidence_level']
        })
    
    def display_verification_report(self, report):
        """Display verification report"""
        print("\n" + "="*70)
        print("🔍 ROBUST AUTHENTICITY VERIFICATION REPORT")
        print("="*70)
        
        status_icon = "✅" if report['authentication_status'] == 'AUTHENTIC' else "❌"
        
        print(f"🕒 Time: {report['timestamp']}")
        print(f"📊 Similarity: {report['similarity_score']:.4f}")
        print(f"🎯 Confidence: {report['confidence_level']}")
        print(f"🔐 Status: {status_icon} {report['authentication_status']}")
        
        if 'copyright_id' in report:
            print(f"📄 Copyright ID: {report['copyright_id']}")
            print(f"👤 Owner: {report['registered_owner']}")
            print(f"✅ Owner Match: {'CONFIRMED' if report['expected_owner_match'] else 'NOT CONFIRMED'}")
        else:
            print("❌ No valid copyright found")
        print("="*70)
    
    def display_database(self):
        """Display database summary"""
        print("\n" + "="*50)
        print("📊 COPYRIGHT DATABASE")
        print("="*50)
        
        if not self.database:
            print("No registrations found")
            return
            
        for copyright_id, data in self.database.items():
            info = data['copyright_info']
            print(f"📄 {copyright_id}")
            print(f"   👤 {info['owner_name']}")
            print(f"   📅 {info['registration_date']}")
            print("   " + "-"*40)

In [4]:
class WatermarkingGUI:
    """Interactive GUI for the watermarking system"""
    
    def __init__(self):
        self.system = CopyrightWatermarkingSystem()
        self.setup_ui()
    
    def setup_ui(self):
        """Setup the user interface"""
        
        # File upload widgets
        self.upload_host = widgets.FileUpload(
            description='Upload Host Image',
            accept='.png,.jpg,.jpeg',
            multiple=False
        )
        
        self.upload_test = widgets.FileUpload(
            description='Upload Test Image',
            accept='.png,.jpg,.jpeg',
            multiple=False
        )
        
        # Copyright information widgets
        self.owner_name = widgets.Text(
            description='Owner Name:',
            placeholder='Enter copyright owner name',
            style={'description_width': '120px'}
        )
        
        self.contact_info = widgets.Text(
            description='Contact Info:',
            placeholder='Email or phone',
            style={'description_width': '120px'}
        )
        
        self.rights_info = widgets.Text(
            description='Rights:',
            value='All rights reserved',
            style={'description_width': '120px'}
        )
        
        # Buttons
        self.register_btn = widgets.Button(
            description='Register Copyright',
            button_style='success',
            icon='copyright'
        )
        
        self.verify_btn = widgets.Button(
            description='Verify Authenticity',
            button_style='info',
            icon='search'
        )
        
        self.summary_btn = widgets.Button(
            description='Database Summary',
            button_style='warning',
            icon='database'
        )
        
        self.log_btn = widgets.Button(
            description='View Verification Log',
            button_style='primary',
            icon='history'
        )
        
        # Output area
        self.output = widgets.Output()
        
        # Connect buttons to functions
        self.register_btn.on_click(self.on_register_click)
        self.verify_btn.on_click(self.on_verify_click)
        self.summary_btn.on_click(self.on_summary_click)
        self.log_btn.on_click(self.on_log_click)
        
        # Layout
        copyright_section = widgets.VBox([
            widgets.HTML("<h3>📝 Copyright Registration</h3>"),
            self.upload_host,
            self.owner_name,
            self.contact_info,
            self.rights_info,
            self.register_btn
        ], layout=widgets.Layout(border='1px solid gray', padding='10px'))
        
        verification_section = widgets.VBox([
            widgets.HTML("<h3>🔍 Authenticity Verification</h3>"),
            self.upload_test,
            self.verify_btn
        ], layout=widgets.Layout(border='1px solid gray', padding='10px'))
        
        info_section = widgets.VBox([
            widgets.HTML("<h3>📊 System Information</h3>"),
            self.summary_btn,
            self.log_btn
        ], layout=widgets.Layout(border='1px solid gray', padding='10px'))
        
        main_layout = widgets.HBox([
            copyright_section,
            verification_section,
            info_section
        ])
        
        display(main_layout)
        display(self.output)
    
    def save_uploaded_image(self, upload_widget, filename):
        """Save uploaded image to temporary file"""
        if not upload_widget.value:
            return None
        
        uploaded = list(upload_widget.value.values())[0]
        with open(filename, 'wb') as f:
            f.write(uploaded['content'])
        
        return filename
    
    def on_register_click(self, btn):
        """Handle copyright registration"""
        with self.output:
            clear_output()
            
            if not self.upload_host.value:
                print("❌ Please upload a host image first")
                return
            
            if not self.owner_name.value:
                print("❌ Please enter owner name")
                return
            
            # Save uploaded image
            host_path = self.save_uploaded_image(self.upload_host, 'temp_host.jpg')
            
            # Register copyright
            try:
                watermarked_img, copyright_info = self.system.register_copyright(
                    host_path,
                    self.owner_name.value,
                    self.contact_info.value,
                    self.rights_info.value
                )
                
                # Display results
                plt.figure(figsize=(12, 5))
                
                # Original image
                original = self.system.preprocess_image(host_path)
                plt.subplot(1, 3, 1)
                plt.imshow(original, cmap='gray')
                plt.title('Original Image')
                plt.axis('off')
                
                # Watermarked image
                plt.subplot(1, 3, 2)
                plt.imshow(watermarked_img, cmap='gray')
                plt.title('Watermarked Image')
                plt.axis('off')
                
                # Difference
                plt.subplot(1, 3, 3)
                difference = np.abs(original - watermarked_img)
                plt.imshow(difference * 20, cmap='hot')  # Amplify difference for visibility
                plt.title('Difference (x20)')
                plt.axis('off')
                plt.colorbar()
                
                plt.tight_layout()
                plt.show()
                
                # Save watermarked image
                watermarked_uint8 = (watermarked_img * 255).astype(np.uint8)
                Image.fromarray(watermarked_uint8).save('watermarked_image.png')
                
                print(f"✅ Copyright registered successfully!")
                print(f"💾 Watermarked image saved as 'watermarked_image.png'")
                print(f"📄 Copyright ID: {copyright_info['copyright_id']}")
                
            except Exception as e:
                print(f"❌ Registration failed: {str(e)}")
                import traceback
                traceback.print_exc()
    
    def on_verify_click(self, btn):
        """Handle authenticity verification"""
        with self.output:
            clear_output()
            
            if not self.upload_test.value:
                print("❌ Please upload an image to verify")
                return
            
            # Save uploaded image
            test_path = self.save_uploaded_image(self.upload_test, 'temp_test.jpg')
            
            try:
                # Verify authenticity
                report, copyright_info, extracted_watermark = self.system.verify_authenticity(
                    test_path, self.owner_name.value if self.owner_name.value else None
                )
                
                # Display report
                self.system.display_verification_report(report)
                
                # Visualize results if watermark was extracted
                if extracted_watermark is not None:
                    plt.figure(figsize=(10, 4))
                    
                    # Test image
                    test_img = self.system.preprocess_image(test_path)
                    plt.subplot(1, 2, 1)
                    plt.imshow(test_img, cmap='gray')
                    plt.title('Test Image')
                    plt.axis('off')
                    
                    # Extracted watermark
                    plt.subplot(1, 2, 2)
                    plt.imshow(extracted_watermark, cmap='gray')
                    plt.title('Extracted Watermark')
                    plt.axis('off')
                    
                    plt.tight_layout()
                    plt.show()
                    
            except Exception as e:
                print(f"❌ Verification failed: {str(e)}")
                import traceback
                traceback.print_exc()
    
    def on_summary_click(self, btn):
        """Display database summary"""
        with self.output:
            clear_output()
            self.system.display_database_summary()
    
    def on_log_click(self, btn):
        """Display verification log"""
        with self.output:
            clear_output()
            self.system.display_verification_log()

In [5]:
# Initialize the system
print("🚀 Initializing Enhanced Copyright Watermarking System...")
gui = WatermarkingGUI()

🚀 Initializing Enhanced Copyright Watermarking System...


NameError: name 'CopyrightWatermarkingSystem' is not defined

In [ ]:
def create_robust_test_images():
    """Create images for robust testing"""
    print("🖼️ Creating robust test images...")
    
    # Create a more textured host image
    x, y = np.meshgrid(np.linspace(0, 8*np.pi, 512), np.linspace(0, 8*np.pi, 512))
    host = (np.sin(x) * np.cos(y) + 0.5 * np.sin(3*x) * np.cos(3*y) + 
            0.3 * np.sin(5*x) * np.cos(5*y) + 2) / 4
    
    # Add realistic texture and noise
    host += np.random.normal(0, 0.03, host.shape)
    host = np.clip(host, 0, 1)
    
    # Save host image
    Image.fromarray((host * 255).astype(np.uint8)).save('robust_host.jpg')
    
    # Create completely different test image
    different = np.random.rand(512, 512) * 0.6 + 0.2
    Image.fromarray((different * 255).astype(np.uint8)).save('robust_different.jpg')
    
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 3, 1)
    plt.imshow(host, cmap='gray')
    plt.title('Robust Host Image')
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.imshow(different, cmap='gray')
    plt.title('Different Test Image')
    plt.axis('off')
    
    # Show watermark pattern example
    system = RobustWatermarkingSystem()
    sample_wm = system.create_robust_watermark('test123', 'Test Company')
    plt.subplot(1, 3, 3)
    plt.imshow(sample_wm, cmap='gray')
    plt.title('Watermark Pattern')
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    return 'robust_host.jpg', 'robust_different.jpg'

# Create test images
host_path, different_path = create_robust_test_images()

In [ ]:
def run_robust_test():
    """Run comprehensive test of the robust system"""
    
    system = RobustWatermarkingSystem(base_alpha=0.25, wavelet='db4')
    
    print("🎯 ROBUST SYSTEM COMPREHENSIVE TEST")
    print("="*60)
    
    # Test 1: Register copyright
    print("\n1. 📝 REGISTER COPYRIGHT")
    watermarked, copyright_info = system.register_copyright(
        host_path, 'Robust Company', 'robust@company.com', 'All rights reserved'
    )
    
    # Save watermarked image
    Image.fromarray((watermarked * 255).astype(np.uint8)).save('robust_watermarked.jpg')
    
    # Display detailed comparison
    original = system.preprocess_image(host_path)
    
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 4, 1)
    plt.imshow(original, cmap='gray')
    plt.title('Original Host')
    plt.axis('off')
    
    plt.subplot(1, 4, 2)
    plt.imshow(watermarked, cmap='gray')
    plt.title('Watermarked')
    plt.axis('off')
    
    plt.subplot(1, 4, 3)
    diff_visual = np.abs(original - watermarked) * 100  # Amplify for visibility
    plt.imshow(diff_visual, cmap='hot', vmin=0, vmax=1)
    plt.title('Difference ×100')
    plt.axis('off')
    plt.colorbar()
    
    plt.subplot(1, 4, 4)
    actual_diff = np.abs(original - watermarked)
    plt.hist(actual_diff.flatten(), bins=50, alpha=0.7, color='blue')
    plt.title('Actual Difference')
    plt.xlabel('Pixel Difference')
    plt.ylabel('Frequency')
    
    plt.tight_layout()
    plt.show()
    
    print(f"📊 Embedding Statistics:")
    print(f"   Max difference: {np.max(actual_diff):.6f}")
    print(f"   Mean difference: {np.mean(actual_diff):.6f}")
    print(f"   PSNR: {-10 * np.log10(np.mean(actual_diff**2)):.2f} dB")
    
    # Test 2: Verify watermarked image (should PASS with high similarity)
    print("\n2. ✅ VERIFY WATERMARKED IMAGE (Should PASS)")
    report1, match1, wm1 = system.verify_authenticity('robust_watermarked.jpg', 'Robust Company')
    system.display_verification_report(report1)
    
    if wm1 is not None:
        original_wm = system.database[copyright_info['copyright_id']]['original_watermark']
        
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 3, 1)
        plt.imshow(original_wm, cmap='gray')
        plt.title('Original Watermark')
        plt.axis('off')
        
        plt.subplot(1, 3, 2)
        plt.imshow(wm1, cmap='gray')
        plt.title('Extracted Watermark')
        plt.axis('off')
        
        plt.subplot(1, 3, 3)
        diff_wm = np.abs(original_wm - wm1)
        plt.imshow(diff_wm, cmap='hot')
        plt.title('Watermark Difference')
        plt.axis('off')
        plt.colorbar()
        
        plt.tight_layout()
        plt.show()
    
    # Test 3: Verify original image (should FAIL with low similarity)
    print("\n3. ❌ VERIFY ORIGINAL IMAGE (Should FAIL)")
    report2, match2, wm2 = system.verify_authenticity(host_path, 'Robust Company')
    system.display_verification_report(report2)
    
    # Test 4: Verify completely different image (should FAIL)
    print("\n4. ❌ VERIFY DIFFERENT IMAGE (Should FAIL)")
    report3, match3, wm3 = system.verify_authenticity(different_path, 'Robust Company')
    system.display_verification_report(report3)
    
    # Test 5: Database summary
    print("\n5. 📊 DATABASE SUMMARY")
    system.display_database()
    
    return system

# Run the robust test
robust_system = run_robust_test()

In [ ]:
def analyze_system_performance():
    """Analyze system performance and robustness"""
    
    print("\n🔬 SYSTEM PERFORMANCE ANALYSIS")
    print("="*50)
    
    # Test various attacks on the watermarked image
    watermarked = robust_system.preprocess_image('robust_watermarked.jpg')
    
    attacks = {
        'Mild Gaussian Noise (σ=0.01)': lambda img: np.clip(img + np.random.normal(0, 0.01, img.shape), 0, 1),
        'Moderate Gaussian Noise (σ=0.02)': lambda img: np.clip(img + np.random.normal(0, 0.02, img.shape), 0, 1),
        'Brightness +15%': lambda img: np.clip(img * 1.15, 0, 1),
        'Contrast +25%': lambda img: np.clip((img - 0.5) * 1.25 + 0.5, 0, 1),
        'JPEG-like Compression': lambda img: (np.round(img * 64) / 64),  # Quantization
    }
    
    print("\n🛡️ ROBUSTNESS AGAINST ATTACKS:")
    print("Attack Type               | Similarity | Status")
    print("-" * 50)
    
    for attack_name, attack_func in attacks.items():
        try:
            attacked = attack_func(watermarked.copy())
            temp_path = f'test_attack_{attack_name.replace(" ", "_").replace("(", "").replace(")", "")}.jpg'
            Image.fromarray((attacked * 255).astype(np.uint8)).save(temp_path)
            
            report, _, _ = robust_system.verify_authenticity(temp_path)
            
            status = "✅" if report['authentication_status'] == 'AUTHENTIC' else "❌"
            print(f"{attack_name:<25} | {report['similarity_score']:.4f}     | {status}")
            
        except Exception as e:
            print(f"{attack_name:<25} | FAILED     | ❌")

# Run performance analysis
analyze_system_performance()

In [ ]:
def final_verification():
    """Final verification that the system works correctly"""
    
    print("\n🎯 FINAL SYSTEM VERIFICATION")
    print("="*50)
    
    # Create a simple test to verify the fundamental principle
    print("Testing fundamental watermarking principle...")
    
    # Get the test data
    copyright_id = list(robust_system.database.keys())[0]
    data = robust_system.database[copyright_id]
    
    # Test 1: Watermarked image should have high similarity
    report1, _, wm1 = robust_system.verify_authenticity('robust_watermarked.jpg')
    sim1 = report1['similarity_score']
    
    # Test 2: Original image should have low similarity  
    report2, _, wm2 = robust_system.verify_authenticity(host_path)
    sim2 = report2['similarity_score']
    
    # Test 3: Different image should have very low similarity
    report3, _, wm3 = robust_system.verify_authenticity(different_path)
    sim3 = report3['similarity_score']
    
    print(f"\n📊 SIMILARITY SCORES:")
    print(f"   Watermarked image: {sim1:.4f} (should be > 0.6)")
    print(f"   Original image:    {sim2:.4f} (should be < 0.3)") 
    print(f"   Different image:   {sim3:.4f} (should be < 0.1)")
    
    # Check if system is working correctly
    is_working = sim1 > 0.6 and sim2 < 0.3 and sim3 < 0.1
    
    print(f"\n🔍 SYSTEM STATUS: {'✅ WORKING CORRECTLY' if is_working else '❌ NEEDS FURTHER FIXES'}")
    
    if is_working:
        print("🎉 The watermarking system is now functioning properly!")
        print("   • Watermarked images are correctly authenticated")
        print("   • Original images are correctly rejected") 
        print("   • Different images are correctly rejected")
    else:
        print("⚠️  The system still needs adjustments")
        if sim2 > 0.3:
            print("   • Original image similarity is too high")
        if sim1 < 0.6:
            print("   • Watermarked image similarity is too low")

# Run final verification
final_verification()